# Autoencoder GPU Test

Notebook for testing class `Autoencoder` features namely:

1. Initialization with parameters
2. Training with `fit` method using the GPU
3. Dimensionality reduction with `encode`

In this example, we will be loading the `creditcardfraud.csv` dataset for testing. The dataset has `29` dimensions. The topology of the model will be `29` for the input layer, `27` for the first hidden layer and `25` for the innermost hidden layer (also known as the latent layer).

In [1]:
# Import necessary libraries and path relative to project
import torch
import pandas as pd

import sys
import os

sys.path.append(os.path.join(os.path.abspath(''), '../pyno/lib'))

from autoencoder import Autoencoder

if not torch.cuda.is_available():
    raise Exception("CUDA not available")
    
# CUDA related information
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION', )
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

__Python VERSION: 3.9.5 (default, May 11 2021, 08:20:37) 
[GCC 10.3.0]
__pyTorch VERSION: 1.9.0+cu111
__CUDA VERSION
/bin/bash: line 1: nvcc: command not found
__CUDNN VERSION: 8005
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0


## Initialization Parameters

### `layers`

An array of integers corresponding to the neuron count from input to innermost latent layer of the autoencoder.

### `h_activation`

The activation function to be used for hidden layers. Possible values:

* `relu`

Default value: `relu`

### `o_activation`

The activation function to be used for the output layer. Possible values:

* `sigmoid`

Default value: `sigmoid`

### `device`

Torch device to be used for training. Default is `torch.device("cpu")` (use CPU of machine)

### `error_type`

The algorithm to be used when computing the overall error for an epoch as well as the `diff` function to determine the residual error between input and output. Possible values:

* `mse`

Default value: `mse`

### `optimizer_type`

The torch optimizer to be used for back propagation. Possible values:

* `adam`

Default value: `adam`

In [2]:
# The topology of the model from input layer to innermost latent layer
layers = [29, 1000, 1000, 15]

h_activation = 'relu'
o_activation = 'sigmoid'
device = torch.device('cuda:0')
error_type = 'mse'
optimizer_type = 'adam'

# Initialize the autoencoder
autoencoder = Autoencoder(
                layers=layers, 
                h_activation=h_activation, 
                o_activation=o_activation, 
                device=device, 
                error_type=error_type, 
                optimizer_type=optimizer_type)

## Loading the Dataset

Loading the dataset involves loading a file to a `pandas` `DataFrame` instance. The dataset to be loaded should be in the form of a CSV file without any headers. To avoid consuming too much memory from a single read for large datasets, provide a `chunk_size` value (integer) to determine how much rows will be loaded to the `DataFrame` per read. The format should be as follows:

```
x1,x2,x3...xn,y
```

where:

`x1,x2,x3...xn` is the multivariate dataset and `y` is the label for that dataset. `y = 1` if the row corresponds to normal data while `y = -1` if the row corresponds to an anomaly.

### Data Segregation

Data will be stripped off the last column to retain only the multivariate data. It will be separated into the variables `positive_data` for all normal data and `negative_data` for all anomalies.

In [3]:
# Instantiate pandas DataFrame
data = pd.DataFrame()

# Chunk size for reading data
chunksize = 10000

# The reference to the dataset. Change this to 
dataset_file = '../data/creditcardfraud.csv'

print("Loading dataset '{}'...".format(dataset_file))

# Read each chunk and append to data frame
for i, chunk in enumerate(pd.read_csv(dataset_file, header=None, chunksize=chunksize)):
    print("Reading chunk %d" % (i + 1))
    data = data.append(chunk)

print("Done loading dataset...")
    
# Check for proper value of input dimensionality to be used by model
input_dim = len(data.columns) - 1
print("Input Dimensionality: %d" % (input_dim))

# Partition the data into positive_data and negative_data
positive_data = data[data[input_dim] == 1].iloc[:,:input_dim]
negative_data = data[data[input_dim] == -1].iloc[:,:input_dim]

# x representing all data regardless of label.
# Need to convert it to a tensor before passing it to the model for training
x = torch.tensor(data.iloc[:,:input_dim].values).float().to(device)

Loading dataset '../data/creditcardfraud.csv'...
Reading chunk 1
Reading chunk 2
Reading chunk 3
Reading chunk 4
Reading chunk 5
Reading chunk 6
Reading chunk 7
Reading chunk 8
Reading chunk 9
Reading chunk 10
Reading chunk 11
Reading chunk 12
Reading chunk 13
Reading chunk 14
Reading chunk 15
Reading chunk 16
Reading chunk 17
Reading chunk 18
Reading chunk 19
Reading chunk 20
Reading chunk 21
Reading chunk 22
Reading chunk 23
Reading chunk 24
Reading chunk 25
Reading chunk 26
Reading chunk 27
Reading chunk 28
Reading chunk 29
Done loading dataset...
Input Dimensionality: 29


## Training with `fit` method

The `fit` method in the `Autoencoder` class takes in the dataset for training using back propogation. The parameters are as follows:

### `x`

The vector of multivariate data representing the training set. No labels should be included. This will also be the value of `y` in terms of autoencoder training.

### `epochs`

Number of iterations for training the entire dataset. Default value is `100`

###  `lr`

The learning rate for back propagtion. Values should be `> 0` and `< 1`. Default value is `0.005`.

### `batch_size`

Number of records to be included in a batch for mini-batch training. Default value is `5`.

In [4]:
epochs = 100
lr = 0.005
batch_size = 10000

autoencoder.fit(
    x, 
    epochs=epochs, 
    lr=lr,
    batch_size=batch_size)

Epoch: 1	Loss: 0.32909
Epoch: 2	Loss: 0.21530
Epoch: 3	Loss: 0.22285
Epoch: 4	Loss: 0.21697
Epoch: 5	Loss: 0.21124
Epoch: 6	Loss: 0.20434
Epoch: 7	Loss: 0.19935
Epoch: 8	Loss: 0.20016
Epoch: 9	Loss: 0.19113
Epoch: 10	Loss: 0.18803
Epoch: 11	Loss: 0.18296
Epoch: 12	Loss: 0.18494
Epoch: 13	Loss: 0.18159


KeyboardInterrupt: 

## Dimensionality Reduction